In [1]:
%matplotlib inline
import rasterio
import matplotlib.pyplot as plt
import gdal
import numpy as np
import timeit

In [2]:
# Function list
def band_average(raster, bands, icethickness):
    band_average = np.zeros((bands.max()+1, 1))
    for n in range(bands.max()+1):
        raster_single = raster.copy()
        raster_single[bands != n] = np.nan
        raster_single[(icethickness == 0) & (emergence_velocity == 0)] = np.nan
        band_average[n,0] = np.nanmean(raster_single)
    return band_average
    

def emergence_pixels(vel_x_raw, vel_y_raw, vel_total, vel_min, vel_depth_avg_factor, icethickness, bands):
    """ Compute the emergence velocity using an ice flux approach
    """
    # Modify vel_y by multiplying velocity by -1 such that matrix operations agree with flow direction
    #    Specifically, a negative y velocity means the pixel is flowing south.
    #    However, if you were to subtract that value from the rows, it would head north in the matrix.
    #    This is due to the fact that the number of rows start at 0 at the top.
    #    Therefore, multipylying by -1 aligns the matrix operations with the flow direction
    vel_y = vel_y_raw * -1 * vel_depth_avg_factor
    vel_x = vel_x_raw * vel_depth_avg_factor
    # Compute the initial volume
    volume_initial = icethickness * pixel_size**2
    # Quality control options:
    # Apply a border based on the max specified velocity to prevent errors associated with pixels going out of bounds
    if option_border == 1:
        border = int(max_velocity / pixel_size)
        for r in range(vel_x.shape[0]):
            for c in range(vel_x.shape[1]):
                if (r < border) | (r >= vel_x.shape[0] - border) | (c < border) | (c >= vel_x.shape[1] - border):
                    vel_x[r,c] = 0
                    vel_y[r,c] = 0
    # Minimum/maximum velocity bounds
    if option_minvelocity == 1:
        vel_x[vel_total < vel_min] = 0
        vel_y[vel_total < vel_min] = 0
    if option_maxvelocity == 1:
        vel_x[vel_total > max_velocity] = 0
        vel_y[vel_total > max_velocity] = 0
    # Remove clusters of high velocity on stagnant portions of glaciers due to feature tracking of ice cliffs and ponds
    if option_stagnantbands == 1:
        vel_x[bands <= stagnant_band] = 0
        vel_y[bands <= stagnant_band] = 0        
    # Compute displacement in units of pixels
    vel_x_pix = vel_x / pixel_size
    vel_y_pix = vel_y / pixel_size
    # Compute the displacement and fraction of pixels moved for all columns (x-axis)
    # col_x1 is the number of columns to the closest pixel receiving ice [ex. 2.6 returns 2, -2.6 returns -2]
    #    int() automatically rounds towards zero
    col_x1 = vel_x_pix.astype(int)
    # col_x2 is the number of columns to the further pixel receiving ice [ex. 2.6 returns 3, -2.6 returns -3]
    #    np.sign() returns a value of 1 or -1, so it's adding 1 pixel away from zero
    col_x2 = (vel_x_pix + np.sign(vel_x_pix)).astype(int)
    # rem_x2 is the fraction of the pixel that remains in the further pixel (col_x2) [ex. 2.6 returns 0.6, -2.6 returns 0.6]
    #    np.sign() returns a value of 1 or -1, so multiplying by that ensures you have a positive value
    #    then when you take the remainder using "% 1", you obtain the desired fraction
    rem_x2 = np.multiply(np.sign(vel_x_pix), vel_x_pix) % 1
    # rem_x1 is the fraction of the pixel that remains in the closer pixel (col_x1) [ex. 2.6 returns 0.4, -2.6 returns 0.4]
    rem_x1 = 1 - rem_x2
    # Repeat the displacement and fraction computations for all rows (y-axis)
    row_y1 = vel_y_pix.astype(int)
    row_y2 = (vel_y_pix + np.sign(vel_y_pix)).astype(int)
    rem_y2 = np.multiply(np.sign(vel_y_pix), vel_y_pix) % 1
    rem_y1 = 1 - rem_y2
    # Compute the mass flux for each pixel
    volume_final = np.zeros(volume_initial.shape)
    for r in range(vel_x.shape[0]):
        for c in range(vel_x.shape[1]):
            volume_final[r+row_y1[r,c], c+col_x1[r,c]] = (
                volume_final[r+row_y1[r,c], c+col_x1[r,c]] + rem_y1[r,c]*rem_x1[r,c]*volume_initial[r,c]
                )
            volume_final[r+row_y2[r,c], c+col_x1[r,c]] = (
                volume_final[r+row_y2[r,c], c+col_x1[r,c]] + rem_y2[r,c]*rem_x1[r,c]*volume_initial[r,c]
                )
            volume_final[r+row_y1[r,c], c+col_x2[r,c]] = (
                volume_final[r+row_y1[r,c], c+col_x2[r,c]] + rem_y1[r,c]*rem_x2[r,c]*volume_initial[r,c]
                )
            volume_final[r+row_y2[r,c], c+col_x2[r,c]] = (
                volume_final[r+row_y2[r,c], c+col_x2[r,c]] + rem_y2[r,c]*rem_x2[r,c]*volume_initial[r,c]
                )
    # Check that mass is conserved (threshold = 0.1 m x pixel_size**2)
    print('Mass is conserved?', (volume_final.sum() - volume_initial.sum()) < 0.1 * pixel_size**2)
    print(volume_final.sum() - volume_initial.sum())
    # Final ice thickness
    icethickness_final = volume_final / pixel_size**2
    # Emergence velocity
    emergence_velocity = icethickness_final - icethickness
    return emergence_velocity


def export_raster(raster, output_filename, ds, NoData_value):
    driver = gdal.GetDriverByName('GTiff')
    new_dataset = driver.Create(output_filename,
                                ds.RasterXSize,    # number of columns
                                ds.RasterYSize,    # number of rows
                                1,                  # number of bands
                                gdal.GDT_Float32)  # datatype of the raster
    new_dataset.SetProjection(ds.GetProjection())
    new_dataset.SetGeoTransform(ds.GetGeoTransform())
    # Now we need to set the band's nodata value to -1
    new_band = new_dataset.GetRasterBand(1)
    new_band.SetNoDataValue(NoData_value)
    # And finally, let's write our NDVI array.
    new_band.WriteArray(raster)
    

def import_raster(raster_fn):
    """Open raster and obtain the values in its first band as an array
    Output: array of raster values
    """
    # open raster dataset
    raster_ds = gdal.Open(raster_fn)
    # extract band information and get values
    raster_band = raster_ds.GetRasterBand(1)
    raster_values = raster_band.ReadAsArray()
    return raster_values

In [3]:
# # ----- Input for Khumbu Glacier ----------
# # Filenames (fn)
# vel_x_fn = '../DShean_20171025/Khumbu_tiffs/Khumbu_20132016_vx_med_ClipV2.tif'
# vel_y_fn = '../DShean_20171025/Khumbu_tiffs/Khumbu_20132016_vy_med_ClipV2.tif'
# vel_x_std_fn = '../DShean_20171025/Khumbu_tiffs/Khumbu_dshean_vx_std_ClipV2.tif'
# vel_y_std_fn = '../DShean_20171025/Khumbu_tiffs/Khumbu_dshean_vy_std_ClipV2.tif'
# icethickness_fn = '../DShean_20171025/Khumbu_tiffs/Khumbu_icethickness_Huss.tif'
# bands_fn = '../DShean_20171025/Khumbu_tiffs/Khumbu_600m_bands_polygons_raster_ClipV2.tif'
# # Additional input
# mc_simulations = 1000
# pixel_size = 30
# min_velocity_mean = 4.267
# min_velocity_std = 2.799 
# max_velocity = 200
# icethickness_uncertainty_percentage = -21
# #  Farinotti et al. [2017] showed on average Huss and Farinotti [2012] underestimate ice thickness by 21%
# vel_depth_avg_factor_min = 0.8
# vel_depth_avg_factor_max = 0.8
# #  Azam et al. (2012) uses 0.9 (state ice flux is +/- 10% based on unknown sliding and other sources of error)
# #  Nuimura et al. (2011) uses 0.8 based on Paterson (1994) and Sakai et al. (2006)
# #  Vincent et al. (2016) uses 0.8
# stagnant_band = 7

# ----- Input for Ngozumpa Glacier ----------
# Filenames
vel_x_fn = '../DShean_20171025/Ngoz_tiffs/ngozumpa_20112016_vx_clippedV2.tif'
vel_y_fn = '../DShean_20171025/Ngoz_tiffs/ngozumpa_20112016_vy_clippedV2.tif'
vel_x_std_fn = '../DShean_20171025/Ngoz_tiffs/Ngoz_dshean_vx_std_clipV2.tif'
vel_y_std_fn = '../DShean_20171025/Ngoz_tiffs/Ngoz_dshean_vy_std_clipV2.tif'
icethickness_fn = '../DShean_20171025/Ngoz_tiffs/Ngoz_IceThickness_Huss.tif'
bands_fn = '../DShean_20171025/Ngoz_tiffs/Ngoz_Bands_600m_ClippedV2.tif'
# Additional input
mc_simulations = 1000
pixel_size = 30
min_velocity_mean = 5.218
min_velocity_std = 4.352
max_velocity = 200
icethickness_uncertainty_percentage = -21
vel_depth_avg_factor_min = 0.8
vel_depth_avg_factor_max = 1.0
stagnant_band = 11

# # ----- Input for Imja-Lhotse Shar Glacier ----------
# # Filenames
# vel_x_fn = '../DShean_20171025/Imja_tiffs/Imja_dshean_vx_annual_clipHuss.tif'
# vel_y_fn = '../DShean_20171025/Imja_tiffs/Imja_dshean_vy_annual_clipHuss.tif'
# vel_x_std_fn = '../DShean_20171025/Imja_tiffs/Imja_dshean_vx_std_annual_clipHuss.tif'
# vel_y_std_fn = '../DShean_20171025/Imja_tiffs/Imja_dshean_vy_std_annual_clipHuss.tif'
# icethickness_fn = '../DShean_20171025/Imja_tiffs/Imja_icethickness_Huss.tif'
# bands_fn = '../DShean_20171025/Imja_tiffs/Imja_600m_Bands_polygons_raster_clipHuss.tif'
# # Additional input
# mc_simulations = 1000
# pixel_size = 30
# min_velocity_mean = 6.205
# min_velocity_std = 9.49
# max_velocity = 200
# icethickness_uncertainty_percentage = -21
# vel_depth_avg_factor_min = 0.8
# vel_depth_avg_factor_max = 0.8
# stagnant_band = 0

# ----- Quality Control Options ----------
# Filter min/max velocity
option_minvelocity = 1
option_maxvelocity = 1
#  > 1 (default) - apply limits specified in model input
#  > 0 - do not apply limits
option_border = 1
#  > 1 (default) - apply the border to prevent errors
#  > 0 - do not apply border, assumes errors near edges are avoided elsewhere
option_stagnantbands = 1
#  > 1 (default) - remove velocities from stagnant bands that are caused by cliffs and ponds
#  > 0 - do not remove the clusters of higher velocities on the tongues of the glaciers

In [4]:
# Open raster for properties (will be used to export tif)
ds = gdal.Open(vel_x_fn)
# Import rasters
vel_x = import_raster(vel_x_fn)
vel_y = import_raster(vel_y_fn)
vel_x_std = import_raster(vel_x_std_fn)
vel_y_std = import_raster(vel_y_std_fn)
icethickness = import_raster(icethickness_fn)
bands = import_raster(bands_fn)

In [5]:
# MONTE CARLO SIMULATIONS OF EMERGENCE VELOCITY
# Generate normal random numbers (mean 0, stdev 1) to include vx and vy uncertainties (actual values calculated in the)
mc_vm_uncertainty = np.random.normal(0, 1, (1, mc_simulations))
# Generate normal distribution for minimum velocity threshold
mc_vmin_uncertainty = np.random.normal(min_velocity_mean, min_velocity_std, (1, mc_simulations))
# Generate uniform distribution for ice thickness uncertainty between Huss and -21% underestimation
mc_icethickness_uncertainty = np.random.uniform(icethickness_uncertainty_percentage, 0, (1, mc_simulations))
# Generate uniform distribution for ratio of surface to mean velocity
mc_vsfc_ratio_uncertainty = np.random.uniform(vel_depth_avg_factor_min, vel_depth_avg_factor_max, (1, mc_simulations))
# Run the Monte Carlo simulations
for sim in range(mc_simulations):  
    print('Simulation:', sim)
    # Compute velocities at each pixel according to uncertainty
    mc_vel_x = vel_x + (mc_vm_uncertainty[0, sim] * vel_x_std)
    mc_vel_y = vel_y + (mc_vm_uncertainty[0, sim] * vel_y_std)
    # Compute the total velocity
    mc_vel_total = (mc_vel_x**2 + mc_vel_y**2)**0.5
    # Minimum velocity including uncertainty
    mc_vmin = mc_vmin_uncertainty[0, sim]
    # Compute ice thicknesses based on uncertainty
    mc_icethickness = icethickness / (1 + mc_icethickness_uncertainty[0, sim]/100)
    #  percent_uncertainty = (x2 - x1) / x1 * 100
    #  percent_uncertainty is the uncertainty (mean = -21%, stdev = 60%)
    #  x2 is the modeled ice thickness (Huss and Farinotti, 2012)
    #  We want to solve for x1, i.e., the ice thickness with the error included
    #    x1 = x2 / (1 + percent_uncertainty / 100)
    mc_vsfc_ratio = mc_vsfc_ratio_uncertainty[0, sim]
    # Emergence Velocity Calculations for each pixel
    emergence_velocity = emergence_pixels(mc_vel_x, mc_vel_y, mc_vel_total, mc_vmin, mc_vsfc_ratio, mc_icethickness, bands)
    # Emergence Velocity Calculations for each band (average the pixel values)
    bands_emergence = band_average(emergence_velocity, bands, icethickness)
    # Record the results
    if sim == 0:
        mc_bands_emergence = bands_emergence
    else:
        mc_bands_emergence = np.concatenate((mc_bands_emergence, bands_emergence), axis=1)
# Calculate statistics of Monte Carlo simulatuions
mc_bands_emergence_wstats = np.zeros((mc_bands_emergence.shape[0],mc_bands_emergence.shape[1]+6))
# Column 01 = band number
bands_col = np.arange(bands.max()+1)
mc_bands_emergence_wstats[:,0] = bands_col
# Column 02 - 06 = median, mean, stdev, min, max
mc_bands_emergence_wstats[:,1] = np.median(mc_bands_emergence, axis=1)
mc_bands_emergence_wstats[:,2] = np.mean(mc_bands_emergence, axis=1)
mc_bands_emergence_wstats[:,3] = np.std(mc_bands_emergence, axis=1)
mc_bands_emergence_wstats[:,4] = np.min(mc_bands_emergence, axis=1)
mc_bands_emergence_wstats[:,5] = np.max(mc_bands_emergence, axis=1)
# Columns 7 - ... = MC simulations
mc_bands_emergence_wstats[:,6:mc_bands_emergence.shape[1]+7] = mc_bands_emergence

Simulation: 0
Mass is conserved? False
565.851854324
Simulation: 1


C:\Users\David\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: RuntimeWarning: Mean of empty slice
  


Mass is conserved? True
-402.359779358
Simulation: 2
Mass is conserved? True
-533.851527214
Simulation: 3
Mass is conserved? False
435.414367676
Simulation: 4
Mass is conserved? False
496.866161346
Simulation: 5
Mass is conserved? False
532.276632309
Simulation: 6
Mass is conserved? True
-650.013784409
Simulation: 7
Mass is conserved? True
-9.71534919739
Simulation: 8
Mass is conserved? False
262.100686073
Simulation: 9
Mass is conserved? True
-301.085806847
Simulation: 10
Mass is conserved? False
379.455770493
Simulation: 11
Mass is conserved? True
-200.297121048
Simulation: 12
Mass is conserved? True
-588.923526764
Simulation: 13
Mass is conserved? True
-366.624988556
Simulation: 14
Mass is conserved? False
277.266485214
Simulation: 15
Mass is conserved? True
-290.618913651
Simulation: 16
Mass is conserved? True
-255.557365417
Simulation: 17
Mass is conserved? False
159.026021004
Simulation: 18
Mass is conserved? True
-617.21768856
Simulation: 19
Mass is conserved? True
-256.11689758

Mass is conserved? False
96.8369445801
Simulation: 154
Mass is conserved? False
1229.63864517
Simulation: 155
Mass is conserved? False
270.770231247
Simulation: 156
Mass is conserved? True
-468.320434093
Simulation: 157
Mass is conserved? True
-478.222425461
Simulation: 158
Mass is conserved? True
-727.140729904
Simulation: 159
Mass is conserved? False
527.801290512
Simulation: 160
Mass is conserved? False
140.373099327
Simulation: 161
Mass is conserved? True
-521.344955444
Simulation: 162
Mass is conserved? True
-532.432220459
Simulation: 163
Mass is conserved? True
-116.195646286
Simulation: 164
Mass is conserved? False
1201.80835629
Simulation: 165
Mass is conserved? True
-717.595737457
Simulation: 166
Mass is conserved? True
-518.04645443
Simulation: 167
Mass is conserved? False
742.866772652
Simulation: 168
Mass is conserved? True
-38.8620419502
Simulation: 169
Mass is conserved? True
26.5889635086
Simulation: 170
Mass is conserved? True
-73.7433214188
Simulation: 171
Mass is cons

Mass is conserved? False
188.723909378
Simulation: 304
Mass is conserved? False
99.1428642273
Simulation: 305
Mass is conserved? False
733.703742981
Simulation: 306
Mass is conserved? False
376.534672737
Simulation: 307
Mass is conserved? False
711.225652695
Simulation: 308
Mass is conserved? True
-455.755164146
Simulation: 309
Mass is conserved? True
-247.280061722
Simulation: 310
Mass is conserved? False
934.495323181
Simulation: 311
Mass is conserved? True
-105.880794525
Simulation: 312
Mass is conserved? False
627.709288597
Simulation: 313
Mass is conserved? False
642.517144203
Simulation: 314
Mass is conserved? True
-174.359972954
Simulation: 315
Mass is conserved? False
174.858728409
Simulation: 316
Mass is conserved? False
345.955926895
Simulation: 317
Mass is conserved? True
-127.589366913
Simulation: 318
Mass is conserved? True
11.8214664459
Simulation: 319
Mass is conserved? False
363.202203751
Simulation: 320
Mass is conserved? True
-254.117074013
Simulation: 321
Mass is con

Mass is conserved? False
283.947226524
Simulation: 454
Mass is conserved? True
-194.788581848
Simulation: 455
Mass is conserved? True
-72.4840078354
Simulation: 456
Mass is conserved? False
450.8585186
Simulation: 457
Mass is conserved? True
-238.407258987
Simulation: 458
Mass is conserved? False
621.826586723
Simulation: 459
Mass is conserved? True
-4.76776790619
Simulation: 460
Mass is conserved? True
-169.994749069
Simulation: 461
Mass is conserved? True
-162.223472595
Simulation: 462
Mass is conserved? True
-413.814439774
Simulation: 463
Mass is conserved? False
678.803049088
Simulation: 464
Mass is conserved? False
395.74606514
Simulation: 465
Mass is conserved? True
-532.989587784
Simulation: 466
Mass is conserved? False
533.125058174
Simulation: 467
Mass is conserved? False
324.70236969
Simulation: 468
Mass is conserved? False
520.223548889
Simulation: 469
Mass is conserved? True
-767.915596962
Simulation: 470
Mass is conserved? True
-149.831718445
Simulation: 471
Mass is conser

Mass is conserved? True
4.53484916687
Simulation: 604
Mass is conserved? True
-154.113894463
Simulation: 605
Mass is conserved? True
-7.62311458588
Simulation: 606
Mass is conserved? True
-196.454399109
Simulation: 607
Mass is conserved? True
-157.485377312
Simulation: 608
Mass is conserved? False
326.74282074
Simulation: 609
Mass is conserved? True
-401.139679909
Simulation: 610
Mass is conserved? False
136.227252483
Simulation: 611
Mass is conserved? True
-103.330726624
Simulation: 612
Mass is conserved? False
293.217420578
Simulation: 613
Mass is conserved? True
48.4791584015
Simulation: 614
Mass is conserved? True
-207.361387253
Simulation: 615
Mass is conserved? True
50.4311275482
Simulation: 616
Mass is conserved? False
351.508448124
Simulation: 617
Mass is conserved? False
130.62651062
Simulation: 618
Mass is conserved? True
-37.740114212
Simulation: 619
Mass is conserved? True
-72.2709331512
Simulation: 620
Mass is conserved? True
-189.085916519
Simulation: 621
Mass is conserve

Mass is conserved? True
-602.219558716
Simulation: 754
Mass is conserved? True
-1087.71245003
Simulation: 755
Mass is conserved? False
209.856185913
Simulation: 756
Mass is conserved? False
243.656914711
Simulation: 757
Mass is conserved? True
-133.975185394
Simulation: 758
Mass is conserved? True
-446.188743591
Simulation: 759
Mass is conserved? True
-121.741167068
Simulation: 760
Mass is conserved? True
59.7796897888
Simulation: 761
Mass is conserved? False
437.213998795
Simulation: 762
Mass is conserved? True
27.2497148514
Simulation: 763
Mass is conserved? False
311.334114552
Simulation: 764
Mass is conserved? False
279.573104858
Simulation: 765
Mass is conserved? False
255.526078224
Simulation: 766
Mass is conserved? True
-1261.55923462
Simulation: 767
Mass is conserved? True
-278.316964149
Simulation: 768
Mass is conserved? False
116.974293709
Simulation: 769
Mass is conserved? True
-791.80623436
Simulation: 770
Mass is conserved? True
-742.921033859
Simulation: 771
Mass is conse

Mass is conserved? False
100.44763422
Simulation: 904
Mass is conserved? True
88.4614019394
Simulation: 905
Mass is conserved? True
-242.57442379
Simulation: 906
Mass is conserved? False
111.631773949
Simulation: 907
Mass is conserved? True
-23.2964000702
Simulation: 908
Mass is conserved? True
-375.807945251
Simulation: 909
Mass is conserved? False
383.846906662
Simulation: 910
Mass is conserved? False
113.953166962
Simulation: 911
Mass is conserved? False
423.736373901
Simulation: 912
Mass is conserved? False
263.125822067
Simulation: 913
Mass is conserved? True
-374.941894531
Simulation: 914
Mass is conserved? True
-242.076710224
Simulation: 915
Mass is conserved? False
308.890901566
Simulation: 916
Mass is conserved? True
-150.592719078
Simulation: 917
Mass is conserved? True
-273.554756165
Simulation: 918
Mass is conserved? True
-191.582016945
Simulation: 919
Mass is conserved? False
191.331928253
Simulation: 920
Mass is conserved? False
114.0007267
Simulation: 921
Mass is conserv

C:\Users\David\Anaconda3\lib\site-packages\numpy\lib\function_base.py:3858: RuntimeWarning: Invalid value encountered in median for 1 results
  r = func(a, **kwargs)


In [6]:
np.savetxt('../Output/emergence_Ngozumpa_mcstat_depthfactorincluded.txt', mc_bands_emergence_wstats)

In [7]:
# # Run without uncertainties
# vel_total = (vel_x**2 + vel_y**2)**0.5
# emergence_velocity = emergence_pixels(vel_x, vel_y, vel_total, icethickness)
# bands_emergence = band_average(emergence_velocity, bands, icethickness)
# print(bands_emergence)

In [8]:
# plt.figure(1)
# plt.imshow(vel_x)
# plt.ylabel('row')
# plt.xlabel('col')
# plt.title('x velocity (m a-1)')
# plt.clim(-120, 60)
# plt.colorbar()

# plt.figure(2)
# plt.imshow(vel_y)
# plt.title('y velocity (m a-1)')
# plt.clim(-120, 60)
# plt.colorbar()

# plt.figure(3)
# plt.imshow(icethickness)
# plt.title('ice thickness (m)')
# plt.colorbar()

# vel_total = total_velocity(vel_x, vel_y)
# plt.figure(4)
# plt.imshow(vel_total)
# plt.ylabel('row')
# plt.xlabel('col')
# plt.title('total velocity (m a-1)')
# plt.clim(-120, 60)
# plt.colorbar()